In [70]:
#import dependencies
import time
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from splinter import Browser
import numpy as np
import random
!which chromedriver

/usr/local/bin/chromedriver


In [71]:
###Cyber News Scrapping

In [72]:
#Set up splinter
try:
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False)
except: 
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=True)

In [73]:
#Create empty lists to store titles, dates, and any other element of interest
article_titles = []
article_dates = []
article_other = []

In [162]:
#Create List of top 10 countries
countries = ["China", "Chinese", "Russia", "Russian", "United States", "US", "USA","American", "Turkish", "Taiwanese", "Brazilian", "Romanian", "Indian", "Italian", "Hungarian"]
top_countries = ["China", "United States", "Turkey", "Russia", "Taiwan", "Brazil", "Romania", "India", "Italy", "Hungary"]

In [74]:
#Function to take current html, get the HTML elements, and use BS to sort through for all the titles and dates
def scrape():
    current_html = browser.html
    soup = BeautifulSoup(current_html, 'html.parser')
    result = soup.body.find_all('div', class_="webpage-item with-thumbnail")
    for each in result:
        article_titles.append(each.find('h3').text)
        article_dates.append(each.find('time').text)

In [75]:
#Splinter visit url to start
url = "https://www.infosecurity-magazine.com/news/"
browser.visit(url)

#Scrapes current page then splinter clicks on button and waits 5-10 seconds for page to load
for x in range(1,10):
    scrape()
    browser.click_link_by_partial_text('Older')
    time.sleep((random.random()*5)+5)
    print(f"{x+1} of 10")


2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10


In [81]:
article_titles


['Clarifai Tries to Clear Up News of Compromised Server',
 'Mexican Campaign Website Attacked with DDoS',
 'Consumers Choose Easy Login over Passwords',
 'Chinese Hackers Target National Datacenter in Watering Hole Spree',
 'NCC Group: Majority of Our Reported Flaws Have Not Been Fixed',
 'Gloucestershire Police BCC Error Leads to £80K Fine',
 '23,000 Individuals Affected in HealthEquity Breach',
 'Cyber-Attacks Expected as World Cup Kicks Off',
 'Military Colleges Aim to Develop Cyber Programs',
 'Apple Update Will Hamper Police Device Crackers',
 'Army Boss: Transport Infrastructure at Risk of Cyber-Attack',
 'European Lawmakers Vote to Ban Kaspersky Lab Products',
 'Clarifai Tries to Clear Up News of Compromised Server',
 'Mexican Campaign Website Attacked with DDoS',
 'Consumers Choose Easy Login over Passwords',
 'Chinese Hackers Target National Datacenter in Watering Hole Spree',
 'NCC Group: Majority of Our Reported Flaws Have Not Been Fixed',
 'Gloucestershire Police BCC Error 

In [83]:
#Splice each word in all the titles
all_words = []
for each in article_titles:
    all_words.append(each.split(" "))

In [88]:
#Unravel lists of words for each title
total_words = [item for sublist in all_words for item in sublist]


In [143]:
#Create dataframe to count occurance of each word
df = pd.DataFrame(total_words, columns= ["Words"])
group = df.groupby("Words")["Words"].count()
group

Words
                 1
#Infosec18:     26
$10m             1
$60              1
&                6
'Predictably     1
2018             1
23,000           3
30%              1
301,000          1
40,000           1
5.9              1
50%              1
92m              1
A                1
AI               2
About            1
Acquisition      2
Across           1
Adobe            1
Adobe,           1
Adult            1
Affected         4
Agile            2
Agility          1
Aim              3
Announced        1
App              1
Apple            4
Apps             1
                ..
Winners          1
Work             2
World            3
Yahoo            1
You              1
Zero-Day         1
a                2
and              1
are              4
as               3
at               3
be               2
by               2
for              6
from             3
in              20
is               3
of              22
on               2
or               1
over             3
the   

In [144]:
#Tally counts for each occurence of each in Country
for x in countries:
    df1 = df.loc[df["Words"] == x]
    print(f"{x} {df1.count()}")

China Words    0
dtype: int64
Chinese Words    4
dtype: int64
Russia Words    0
dtype: int64
Russian Words    1
dtype: int64
United States Words    0
dtype: int64
US Words    3
dtype: int64
USA Words    0
dtype: int64
American Words    0
dtype: int64
Turkish Words    0
dtype: int64
Taiwanese Words    0
dtype: int64
Brazilian Words    0
dtype: int64
Romanian Words    0
dtype: int64
Indian Words    0
dtype: int64
Italian Words    0
dtype: int64
Hungarian Words    0
dtype: int64


In [153]:
#Tallied top counts
top_counts = [4,3,0,1,0,0,0,0,0,0] 
top_counts

[4, 3, 0, 1, 0, 0, 0, 0, 0, 0]

In [212]:
#Create dataframe for top occurance of top countries
df2 = pd.DataFrame(top_countries, columns=['country'])
df2['count'] = top_counts
df2

,country,count
0,China,4
1,United States,3
2,Turkey,0
3,Russia,1
4,Taiwan,0
5,Brazil,0
6,Romania,0
7,India,0
8,Italy,0
9,Hungary,0


In [213]:
#convert to dictionary for SQL Alchemy
dic = df2.to_dict(orient='records')

In [214]:
dic

[{'count': 4, 'country': 'China'},
 {'count': 3, 'country': 'United States'},
 {'count': 0, 'country': 'Turkey'},
 {'count': 1, 'country': 'Russia'},
 {'count': 0, 'country': 'Taiwan'},
 {'count': 0, 'country': 'Brazil'},
 {'count': 0, 'country': 'Romania'},
 {'count': 0, 'country': 'India'},
 {'count': 0, 'country': 'Italy'},
 {'count': 0, 'country': 'Hungary'}]

In [164]:
### Add to SQLITE

# Import SQL Alchemy dependencies
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Float, Text, Numeric
from sqlalchemy import inspect

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [173]:
#create connection
engine = create_engine('sqlite:///../db/apache.sqlite')
Base.metadata.create_all(engine)

In [211]:
#create classes
Base = declarative_base()

class NewsCountries(Base):
    __tablename__ = 'news_countries'
    
    id = Column(Integer, primary_key=True)
    count = Column(Integer) 
    country = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.news_countries}"

In [207]:
conn = engine.connect()

In [215]:
#Add top country into table
conn.execute(NewsCountries.__table__.delete())
conn.execute(NewsCountries.__table__.insert(), dic)

In [216]:
#Query to double check
engine.execute("select * from news_countries").fetchall()

[(1, 'China', 4),
 (2, 'United States', 3),
 (3, 'Turkey', 0),
 (4, 'Russia', 1),
 (5, 'Taiwan', 0),
 (6, 'Brazil', 0),
 (7, 'Romania', 0),
 (8, 'India', 0),
 (9, 'Italy', 0),
 (10, 'Hungary', 0)]

In [189]:
#Check that SQLite file has new table
engine.table_names()

['access_response', 'news_countries']